# 1. To SingleCellExperiment

## load h5ad data

In [1]:
library(anndata)

In [2]:
adata <- read_h5ad("./Placenta.h5ad")

In [3]:
adata

AnnData object with n_obs × n_vars = 9845 × 42117
    obs: 'cid', 'seq_tech', 'donor_ID', 'donor_gender', 'donor_age', 'donor_status', 'original_name', 'organ', 'region', 'subregion', 'sample_status', 'treatment', 'ethnicity', 'cell_type', 'cell_id', 'study_id', 'nCount_RNA', 'nFeature_RNA'
    var: 'Gene_symbol', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'cell_type_colors', 'hvg', 'study_id_colors'
    obsm: 'X_scANVI_umap', 'X_umap'

## convert into SingleCellExperiment

In [4]:
library(SingleCellExperiment)

Loading required package: SummarizedExperiment

Loading required package: MatrixGenerics

Loading required package: matrixStats


Attaching package: ‘MatrixGenerics’


The following objects are masked from ‘package:matrixStats’:

    colAlls, colAnyNAs, colAnys, colAvgsPerRowSet, colCollapse,
    colCounts, colCummaxs, colCummins, colCumprods, colCumsums,
    colDiffs, colIQRDiffs, colIQRs, colLogSumExps, colMadDiffs,
    colMads, colMaxs, colMeans2, colMedians, colMins, colOrderStats,
    colProds, colQuantiles, colRanges, colRanks, colSdDiffs, colSds,
    colSums2, colTabulates, colVarDiffs, colVars, colWeightedMads,
    colWeightedMeans, colWeightedMedians, colWeightedSds,
    colWeightedVars, rowAlls, rowAnyNAs, rowAnys, rowAvgsPerColSet,
    rowCollapse, rowCounts, rowCummaxs, rowCummins, rowCumprods,
    rowCumsums, rowDiffs, rowIQRDiffs, rowIQRs, rowLogSumExps,
    rowMadDiffs, rowMads, rowMaxs, rowMeans2, rowMedians, rowMins,
    rowOrderStats, rowProds, rowQuantiles, rowRanges

In [5]:
# create sce object
mat <- t(adata$X) 

sce <- SingleCellExperiment(
  assays = list(counts = mat),
  colData = adata$obs,
  rowData = adata$var
)

In [6]:
# Add obsm data into reducedDims
for (nm in names(adata$obsm)) {
  mat <- adata$obsm[[nm]]
  if (is.matrix(mat) || is.data.frame(mat)) {
    reducedDims(sce)[[nm]] <- as.matrix(mat)
  }
}

In [7]:
sce

class: SingleCellExperiment 
dim: 42117 9845 
metadata(0):
assays(1): counts
rownames(42117): A1BG A1BG-AS1 ... ZZEF1 ZZZ3
rowData names(5): Gene_symbol highly_variable means dispersions
  dispersions_norm
colnames(9845): ChorionicVillus_1.ACCTGATACTTCCGTGGC_44
  ChorionicVillus_1.TGATCAGGGTTTCGGCAG_44 ...
  ChorionicVillus_1.CCGACGTCGTAAAAGCGG_44
  ChorionicVillus_1.GAACGCTTCCGCAACCTA_44
colData names(18): cid seq_tech ... nCount_RNA nFeature_RNA
reducedDimNames(2): X_scANVI_umap X_umap
mainExpName: NULL
altExpNames(0):

## saving

In [8]:
saveRDS(sce, file = "./Placenta_SCE.rds")

# 2. To Seurat object

## load h5ad data

In [1]:
library(anndata)

In [2]:
adata <- read_h5ad("./Placenta.h5ad")

In [14]:
adata

AnnData object with n_obs × n_vars = 9845 × 42117
    obs: 'cid', 'seq_tech', 'donor_ID', 'donor_gender', 'donor_age', 'donor_status', 'original_name', 'organ', 'region', 'subregion', 'sample_status', 'treatment', 'ethnicity', 'cell_type', 'cell_id', 'study_id', 'nCount_RNA', 'nFeature_RNA'
    var: 'Gene_symbol', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'cell_type_colors', 'hvg', 'study_id_colors'
    obsm: 'X_scANVI_umap', 'X_umap'

## convert into Seurat

In [15]:
library(Seurat)

In [ ]:
# Get expression
logmat <- t(adata$X)
logmat <- as(logmat, "dgCMatrix")

gene_names <- rownames(adata$var)
rownames(logmat) <- gene_names

cell_names <- rownames(adata$obs)
colnames(logmat) <- cell_names
# Get metadata
meta.data <- as.data.frame(adata$obs)
# Get gene information
features <- as.data.frame(adata$var)

# Create Seurat object
seu <- CreateSeuratObject(counts = logmat,
                          meta.data = meta.data)

DefaultAssay(seu) <- "RNA"
seu[["RNA"]] <- SetAssayData(
  object = seu[["RNA"]],
  slot = "data",
  new.data = logmat
)

rownames(seu) <- rownames(features)

In [25]:
for (key in names(adata$obsm)) {
    coords <- as.matrix(adata$obsm[[key]])
    if (nrow(coords) == ncol(seu)) {  
      rownames(coords) <- colnames(seu)
        
      red_name <- gsub("^X_", "", key)
      seu[[tolower(red_name)]] <- CreateDimReducObject(
        embeddings = coords,
        key = paste0(toupper(red_name), "_"),
        assay = DefaultAssay(seu)
      )
    }
  }

Warning message:
“Keys should be one or more alphanumeric characters followed by an underscore, setting key from SCANVI_UMAP_ to SCANVIUMAP_”
Warning message:
“No columnames present in cell embeddings, setting to 'SCANVIUMAP_1:2'”
Warning message:
“No columnames present in cell embeddings, setting to 'UMAP_1:2'”


In [26]:
seu

An object of class Seurat 
42117 features across 9845 samples within 1 assay 
Active assay: RNA (42117 features, 0 variable features)
 2 layers present: counts, data
 2 dimensional reductions calculated: scanvi_umap, umap

In [27]:
saveRDS(seu, file = "./Placenta_seurat.rds")

# 3. To loom

Use these codes in python

In [ ]:
import scanpy as sc

adata = sc.read_h5ad("./Placenta.h5ad")
adata.write_loom("./Placenta.loom", write_obsm_varm=True)